# GA parameters

In [446]:
# GA parameter sets: baseline, exploration and exploitation.
GA_PARAMETER_SETS = [
    {
        "name": "Baseline set",
        "generation_number": 4,
        "crossover_probability": 0.8,
        "mutation_probability": 0.1
    },
    {
        "name": "Exploration scenario",
        "generation_number": 100,
        "crossover_probability": 0.9,
        "mutation_probability": 0.2
    },
    {
        "name": "Exploitation scenario",
        "generation_number": 30,
        "crossover_probability": 0.7,
        "mutation_probability": 0.001
    }
]


# dictionary for each scenario with instance 1 and two for all scenarios (small, medium and large)
PROBLEM_INSTANCES = [
    {
        "name": "Small instance 1",
        "population_size": 14,
        "num_vehicles": 3,
    },
    {
        "name": "Small instance 2",
        "population_size": 20,
        "num_vehicles": 10,
    },
    {
        "name": "Medium instance 1",
        "population_size": 21,
        "num_vehicles": 10,
    },
    {
        "name": "Medium instance 2",
        "population_size": 28,
        "num_vehicles": 20,
    },
    {
        "name": "Large instance 1",
        "population_size": 32,
        "num_vehicles": 20,
    },
    {
        "name": "Large instance 2",
        "population_size": 50,
        "num_vehicles": 25,
    }
]

# Functions for genetic algorithm

# Genetic algorithm

##### Create locations data set:

In [ ]:
# 1: Import customers from example CSV.

import pandas as pd

# path to customers list
csv_file_path = 'customers.csv'

# create pandas dataframe with all customers from CSV-file
CUSTOMERS_DF = pd.read_csv(csv_file_path)
print("CSV loaded successfully. Number of customers: ", CUSTOMERS_DF.shape[0])
print("First 5 customers: ", CUSTOMERS_DF.head(5))

# Analysis and results

# 1. Create population

#### 1.1 Import customer list from CSV

In [447]:
import pandas as pd

# path to customers list
csv_file_path = 'customers.csv'

# create pandas dataframe with all customers from CSV-file
CUSTOMERS_DF = pd.read_csv(csv_file_path)
print("CSV loaded successfully. Number of customers: ", CUSTOMERS_DF.shape[0])
print("First 5 customers: ", CUSTOMERS_DF.head(5))

CSV loaded successfully. Number of customers:  50
First 5 customers:     id          x          y
0   1  63.942680   2.501076
1   2  27.502932  22.321074
2   3  73.647121  67.669949
3   4  89.217957   8.693883
4   5  42.192182   2.979722


#### 1.2 Select random population of 14 customers

In [448]:
"""
This section selects 14 random customers from the pandas dataframe.
random_state is set to 42 to ensure reproducibility.
"""

# select customers randomly from customers_list_df to match needed population size.
customer_population = customers_list_df.sample(n=POPULATION_SIZE, random_state=42)
print("First 5 customers:")
print(customer_population.head(5))

# list of customer ids of selected customers
customer_ids = customer_population['id'].tolist()
print("----------------------------")
print("List of ids of selected customers:", customer_ids)

First 5 customers:
    id          x          y
13  14   9.274584   9.671638
39  40  26.488017  24.662751
30  31  98.952335  63.999976
45  46  10.964913  62.744604
17  18  37.853438  55.204063
----------------------------
List of ids of selected customers: [14, 40, 31, 46, 18, 49, 27, 26, 33, 20, 13, 5, 38, 9]


#### 1.3 Define generations, depot and vehicles.

In [449]:
# number of generations and vehicles
VEHICLES = 3

# coordinated for depot
DEPOT_LOC = [50,50]

# marker of depot it chromosome list
DEPOT_MARKER = 0

MUTATION_RATE = 10

# list of internal depot markers, with the constraint that all vehicles start and finish at depot (0)
num_internal_depot_markers = VEHICLES - 1

#### 1.4 Create initial random chromosome.

In [461]:
import random

# create random first chromosome
def create_individual():
    """
    Creates the first random individual chromosome by shuffling customer ids.
    """

    random.shuffle(customer_ids)

    chromosome = customer_ids

    return chromosome

print("Creating individual chromosome...")
chromosome = create_individual()
print(chromosome)

Creating individual chromosome...
[27, 14, 18, 49, 26, 40, 33, 20, 38, 5, 31, 13, 46, 9]


#### 1.5 Create dataframe with all location variables. Depot, and customers x and y.

In [451]:
coordinates_df = pd.concat(
    [customer_population,
     pd.DataFrame([[0] + DEPOT_LOC], columns=customer_population.columns)],
)

print(coordinates_df)

    id          x          y
13  14   9.274584   9.671638
39  40  26.488017  24.662751
30  31  98.952335  63.999976
45  46  10.964913  62.744604
17  18  37.853438  55.204063
48  49  99.612138  52.911435
26  27  26.697782  93.665459
25  26  37.018097  20.950703
32  33  84.285192  77.599991
19  20  86.170690  57.735215
12  13  95.721307  33.659455
4    5  42.192182   2.979722
37  38  65.543867  39.563190
8    9  22.044062  58.926568
0    0  50.000000  50.000000


#### 1.6 Function to add depot locations

In [452]:
def add_vehicles(individual):
    """
    Adds number of vehicles to individual chromosome by adding trips to depot.
    """

    print("Number of depot markers: ", num_internal_depot_markers)
    print("Individual chromosome:")
    print(individual)

    print("Placing inner depot markers...")

    # place inner depot markers
    positions = list(range(1, len(individual)))         # possible placements for inner depot markers (not start, end or adjacent)
    individual_with_vehicles = individual.copy()        # new chromosome, ready to add depot markers into

    # shuffle possible positions of inner depot markers
    random.shuffle(positions)

    zeros_indices = []
    for pos in positions:
        if all(abs(pos - z) > 1 for z in zeros_indices):
            zeros_indices.append(pos)
            if len(zeros_indices) == num_internal_depot_markers:
                break

    for idx in sorted(zeros_indices, reverse=True):
        individual_with_vehicles.insert(idx, 0)


    print("Adding depot markers at beginning and end...")
    # create chromosome which start at 0, and end at 0, with the customers and random point of depot-visits inbetween
    individual_with_vehicles = [DEPOT_MARKER] + individual_with_vehicles + [DEPOT_MARKER]

    print("Complete individual with vehicles: ", individual_with_vehicles)

    return individual_with_vehicles

add_vehicles(chromosome)

Number of depot markers:  2
Individual chromosome:
[46, 27, 14, 33, 38, 13, 40, 49, 9, 5, 26, 31, 20, 18]
Placing inner depot markers...
Adding depot markers at beginning and end...
Complete individual with vehicles:  [0, 46, 27, 0, 14, 33, 38, 13, 40, 49, 9, 5, 26, 0, 31, 20, 18, 0]


[0, 46, 27, 0, 14, 33, 38, 13, 40, 49, 9, 5, 26, 0, 31, 20, 18, 0]

# 2. Fitness function

#### 2.1 Function to calculate the Euclidean distances between two objects

In [453]:
import numpy as np

def calculate_euclidean_distance(id1, id2, loc_df):
    """
    Calculates the Euclidean distance between two points.

    Based on formula: sqrt((x1 - x2)^2 + (y1 - y2)^2)

    :param id1: the id of first location
    :param id2: the id of second location
    :param loc_df: the dataframe containing the coordinates to all locations
    """

    # find x and y coordinated for id1
    x1 = loc_df.loc[loc_df['id'] == id1]['x'].iloc[0]
    y1 = loc_df.loc[loc_df['id'] == id1]['y'].iloc[0]

    # find x and y coordinates for id2
    x2 = loc_df.loc[loc_df['id'] == id2]['x'].iloc[0]
    y2 = loc_df.loc[loc_df['id'] == id2]['y'].iloc[0]

    # Euclidean calculation on objects id1 and id2
    return np.sqrt((x1 - x2) ** 2 + (y1 - y2) ** 2)

#### 2.2 Fitness function - evaluates fitness of a single chromosome

In [454]:
def calculate_fitness(individual):
    """
    Calculate the fitness of the individual chromosome.

    :param individual:

    :return: the fitness of the individual chromosome (value between 0 and 1)
    """
    routes_segmented = []
    routes_distances = []
    start_index_of_route = 0

    try:
        # separate all vehicle trips into individual routes (list)
        # iterate through chromosome to find depot-markers and segment into individual vehicle routes.
        for i in range(len(individual)):

            if individual[i] == depot_marker and i != 0:                    # avoid counting starting depot point
                route_segment = individual[start_index_of_route: i + 1]     # create a temporary route segment for current vehicle trip
                routes_segmented.append(route_segment)                            # append list of all routes with new route segment

                # start to look for new route from where the last ended at the depot
                start_index_of_route = i
                print("The route segmented is: ", route_segment)

        # calculate distances for each trip
        for route in routes_segmented:

            # temporary list of individual Euclidean distances in one route segment
            single_route_distance = []

            print("processing new route...")
            for loc1,loc2 in zip(route, route[1:]):
                euclidean_distance = calculate_euclidean_distance(loc1, loc2, coordinates_df)
                single_route_distance.append(euclidean_distance)

            total_route_distance = sum(single_route_distance)
            print("calculated distance: ", total_route_distance)
            routes_distances.append(total_route_distance)


        # sum distance of all trips
        total_accumulated_distance = sum(routes_distances)

        # calculate fitness based on total_trip_distance and return value
        return 1 / total_accumulated_distance

    except ZeroDivisionError:
        print("There are no routes to segment. Check if depot markers are present.")
        print("The route segmented is: ", individual)
        print("Number of depot markers present: ", individual.count(0))

calculate_fitness(chromosome)

There are no routes to segment. Check if depot markers are present.
The route segmented is:  [46, 27, 14, 33, 38, 13, 40, 49, 9, 5, 26, 31, 20, 18]
Number of depot markers present:  0


## 3. Crossover and mutation

#### 3.1 Crossover function

In [455]:
def crossover_pmx(parent1, parent2):
    """

    :param parent1:
    :param parent2:
    """
    length = len(parent1)
    child1 = [-1] * length
    child2 = [-1] * length

    print("step 1:")
    print("parent1: ", parent1)
    print("parent2: ", parent2)

    if random.random() < CROSSOVER_RATE:           # runs crossover if the randomly selected float is lower than the crossover rate

        # define crossover start and end
        start, end = sorted(random.sample(range(length), 2))

        print("--------------------------------------")
        print("the crossover start and end: ", start, ", ", end)

        #
        child1[start:end] = parent1[start:end]
        child2[start:end] = parent2[start:end]

        print("--------------------------------------")
        print("step 2:")
        print("child1: ", child1)
        print("child2: ", child2)

        # mapping between the parents and children. Maps duplicates
        mapping1_to_2 = {} # maps elements from segments of parent1 to parent2
        mapping2_to_1 = {} # Maps elements from segments of parent2 to parent1
        for i in range(start, end):
            gene1 = parent1[i]
            gene2 = parent2[i]
            mapping1_to_2[gene1] = gene2
            mapping2_to_1[gene2] = gene1

        print("--------------------------------------")
        print("step 3:")
        print("mapping 1 elements from segment of parent 1 to parent 2: ", mapping1_to_2)
        print("mapping 2 elements from segment of parent 2 to parent 1: ", mapping2_to_1)

        # 4. Fill the remaining positions in offspring1
        # Iterate through parent1. For each gene, if it's not already in offspring1,
        # place it. If it causes a duplicate, use the mapping to find a non-duplicate.
        for i in range(length):
            if start <= i < end:
                continue # Segment already copied

            gene_to_place = parent2[i]
            # Check if the gene is already in offspring1 (i.e., it's part of the copied segment from parent2)
            while gene_to_place in child1[start:end]:
                gene_to_place = mapping1_to_2[gene_to_place] # Follow the mapping

            # Place the non-duplicate gene
            child1[i] = gene_to_place

        for i in range(length):
            if start <= i < end:
                continue # Segment already copied

            gene_to_place = parent1[i]
            # Check if the gene is already in offspring1 (i.e., it's part of the copied segment from parent2)
            while gene_to_place in child2[start:end]:
                gene_to_place = mapping2_to_1[gene_to_place] # Follow the mapping

            # Place the non-duplicate gene
            child2[i] = gene_to_place

    return child1, child2

new_child1, new_child2 = crossover_pmx(chromosome, [33, 46, 27, 13, 9, 26, 5, 31, 38, 14, 18, 40, 20, 49])

print("")
print("----------- Newborn children: -----------")
print("Crossed child1: ", new_child1)
print("Crossed child2: ", new_child2)


step 1:
parent1:  [46, 27, 14, 33, 38, 13, 40, 49, 9, 5, 26, 31, 20, 18]
parent2:  [33, 46, 27, 13, 9, 26, 5, 31, 38, 14, 18, 40, 20, 49]

----------- Newborn children: -----------
Crossed child1:  [-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1]
Crossed child2:  [-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1]


#### 3.2 Mutation function

In [456]:
def mutate(individual):
    """
    Mutates an individual chromosome.

    :param individual: individual chromosome (list of customers)
    :return: the mutated chromosome (list of customers with slight mutations)
    """

    if random.random() < MUTATION_RATE:           # randomly select a number and runs code if it is less than mutation rate

        # mutation procedure
        pos1, pos2 = random.sample(range(len(individual)), 2)                       # selects two distinct random positions in list
        individual[pos1], individual[pos2] = individual[pos2], individual[pos1]     # swap elements in positions

    return individual

print(mutate(new_child1))


[-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1]


## 4. Genetic algorithm

In [457]:
def visualize_individual(individual, fitness_value):
    """
    Creates a string representation of an individual.

    Args:
    individual (list): A tour represented as a list of city indices.
    fitness_value (float): The fitness value of the individual.

    Returns:
    str: A string representation of the tour and its fitness.
    """
    route = " -> ".join(map(str, individual))
    return f"{route} Fitness: {fitness_value:.6f}"

In [458]:
def genetic_algorithm():
    """
    Runs the genetic algorithm for the Traveling Salesman Problem.

    This function initializes a population, evolves it over several generations,
    and prints the state of the population at each step.
    """
    population = [create_individual() for _ in range(POPULATION_SIZE)]

    for generation in range(GENERATIONS):
        print(f"\nGeneration {generation}:")
        for i, ind in enumerate(population):
            print(f"Individual {i}: {visualize_individual(ind, calculate_fitness(ind))}")

        new_population = []

        print("\nCrossover and Mutation:")
        for i in range(0, POPULATION_SIZE, 2):
            parent1, parent2 = random.sample(population, 2)
            child1, (start, end) = crossover_pmx(parent1, parent2)
            child2, _ = crossover_pmx(parent2, parent1)

            print(f"\nParent 1: {visualize_individual(parent1, calculate_fitness(parent1))}")
            print(f"Parent 2: {visualize_individual(parent2, calculate_fitness(parent2))}")
            print(f"Crossover range: {start} to {end}")
            print(f"Child 1:  {visualize_individual(child1, calculate_fitness(child1))}")

            mutated_child1 = mutate(child1)
            print(f"Mutated: {visualize_individual(mutated_child1, calculate_fitness(mutated_child1))}")

            new_population.extend([mutated_child1, mutate(child2)])

        population = new_population

    print("\nFinal Population:")
    for i, ind in enumerate(population):
        print(f"Individual {i}: {visualize_individual(ind, calculate_fitness(ind))}")

if __name__ == "__main__":
    genetic_algorithm()


Generation 0:
There are no routes to segment. Check if depot markers are present.
The route segmented is:  [33, 5, 31, 18, 13, 49, 46, 9, 38, 26, 40, 14, 27, 20]
Number of depot markers present:  0


TypeError: unsupported format string passed to NoneType.__format__